# %% [markdown]
# # 🎗️ BREAST CANCER PREDICTION PIPELINE
# ## Complete ML System - From Data to Deployment
# 
# **Objective:** Predict breast cancer (Malignant/Benign) with 95%+ accuracy
# **Dataset:** Wisconsin Breast Cancer Database
# **Features:** 30 cell nucleus characteristics (radius, texture, perimeter, area, smoothness, etc.)
# **Algorithms:** SVM, Logistic Regression, Random Forest, XGBoost
# **Author:** Your Name | **Date:** 2024

In [ ]:
# %% [markdown]
# ## 1. Environment Setup

# %%
import sys, os
sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.config.settings import ProjectConfig
from src.utils.logger import setup_logging

config = ProjectConfig.load("cancer")
logger = setup_logging("cancer_pipeline")
logger.info("✅ Breast cancer prediction pipeline initialized")
print(f"✅ Working directory: {os.getcwd()}")

In [ ]:
# %% [markdown]
# ## 2. Data Loading & Validation

# %%
from src.data.loader import DataLoader
from src.data.validator import DataValidator

loader = DataLoader(config)
df = loader.load_breast_cancer()
validator = DataValidator(config)
report = validator.generate_quality_report(df)
print(f"✅ Dataset loaded: {df.shape[0]} patients, {df.shape[1]} features")
print(f"📊 Features: Mean radius, texture, perimeter, area, smoothness, ...")

In [ ]:
# %% [markdown]
# ## 3. Initial Data Inspection

# %%
print("📊 FIRST 5 ROWS (first 10 features):")
display(df.iloc[:, :10].head())

print("\n📊 DATA INFO:")
print(df.info())

print("\n📊 BASIC STATISTICS (first 5 features):")
display(df.iloc[:, :5].describe().round(2))

In [ ]:
# %% [markdown]
# ## 4. Missing Values Analysis

# %%
missing = pd.DataFrame({
    'Column': df.columns,
    'Missing': df.isnull().sum().values,
    'Percentage': (df.isnull().sum() / len(df) * 100).values
}).sort_values('Percentage', ascending=False)

print("🔍 MISSING VALUES REPORT:")
if missing['Missing'].sum() > 0:
    display(missing[missing['Missing'] > 0])
else:
    print("✅ No missing values found!")

print(f"\n📊 Total missing: {df.isnull().sum().sum()} cells")
print(f"📊 Complete cases: {len(df.dropna())}/{len(df)}")

In [ ]:
# %% [markdown]
# ## 5. Duplicate Check

# %%
duplicates = df.duplicated().sum()
print(f"📊 DUPLICATE ROWS: {duplicates}")
print(f"Percentage: {(duplicates/len(df))*100:.2f}%")

if duplicates > 0:
    print("⚠️ Duplicates found - will remove during preprocessing")
else:
    print("✅ No duplicates found")

In [ ]:
# %% [markdown]
# ## 6. Target Variable Distribution

# %%
target_dist = df['target'].value_counts()
target_pct = df['target'].value_counts(normalize=True) * 100

print("🎯 TARGET DISTRIBUTION:")
print(f"Benign:     {target_dist[0]} ({target_pct[0]:.1f}%)")
print(f"Malignant:  {target_dist[1]} ({target_pct[1]:.1f}%)")

if target_pct.min() < 30:
    print("⚠️ Mildly imbalanced - may apply SMOTE if needed")

In [ ]:
# %% [markdown]
# ## 7. Data Types Analysis

# %%
dtype_df = pd.DataFrame({
    'Column': df.columns[:10],  # First 10 columns
    'Type': df.dtypes.values[:10],
    'Unique': [df[col].nunique() for col in df.columns[:10]]
})

print("📊 DATA TYPES (first 10 columns):")
display(dtype_df)

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\n✅ All {len(numeric_cols)} columns are numeric")

In [ ]:
# %% [markdown]
# ## 8. Feature Groups Understanding

# %%
feature_groups = {
    'mean': [col for col in df.columns if 'mean' in col.lower() or '_1' in col],
    'se': [col for col in df.columns if 'se' in col.lower() or '_2' in col],
    'worst': [col for col in df.columns if 'worst' in col.lower() or '_3' in col]
}

print("📊 FEATURE GROUPS:")
for group, cols in feature_groups.items():
    print(f"\n{group.upper()} features ({len(cols)}):")
    print(f"   {', '.join(cols[:3])}...")

print("\n📌 Note: Features are measurements of cell nuclei:")
print("   - Mean: Average value")
print("   - SE: Standard Error")
print("   - Worst: Largest value")

In [ ]:
# %% [markdown]
# ## 9. Statistical Summary by Feature Group

# %%
for group_name, cols in feature_groups.items():
    if cols:
        print(f"\n📈 {group_name.upper()} FEATURES SUMMARY:")
        group_stats = df[cols].describe().T[['mean', 'std', 'min', 'max']].round(2)
        display(group_stats.head())

In [ ]:
# %% [markdown]
# ## 10. Target Distribution Visualization

# %%
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

bars = ax1.bar(['Benign', 'Malignant'], target_dist.values, 
               color=['#4ECDC4', '#FF6B6B'])
ax1.set_title('Breast Cancer Target Distribution', fontweight='bold', fontsize=14)
ax1.set_ylabel('Count')
for bar, val in zip(bars, target_dist.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height()+5, 
             str(val), ha='center', fontweight='bold')

ax2.pie(target_dist.values, labels=['Benign', 'Malignant'], 
        autopct='%1.1f%%', colors=['#4ECDC4', '#FF6B6B'],
        explode=(0, 0.05), startangle=90)
ax2.set_title('Target Distribution (%)', fontweight='bold', fontsize=14)

plt.suptitle('Breast Cancer Dataset Overview', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 11. Mean Radius Analysis (Feature 1)

# %%
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Distribution by target
ax = axes[0, 0]
for target in [0, 1]:
    subset = df[df['target'] == target]['feature_1']
    ax.hist(subset, alpha=0.7, label=f'Target {target}', bins=20,
            color=['#4ECDC4', '#FF6B6B'][target])
ax.set_xlabel('Mean Radius')
ax.set_ylabel('Frequency')
ax.set_title('Mean Radius Distribution', fontweight='bold')
ax.legend(['Benign', 'Malignant'])

# Boxplot
ax = axes[0, 1]
df.boxplot(column='feature_1', by='target', ax=ax)
ax.set_title('Mean Radius Boxplot', fontweight='bold')
ax.set_xlabel('Tumor Type')
ax.set_ylabel('Mean Radius')
ax.set_xticklabels(['Benign', 'Malignant'])

# Violin plot
ax = axes[1, 0]
sns.violinplot(x='target', y='feature_1', data=df, ax=ax,
               palette=['#4ECDC4', '#FF6B6B'])
ax.set_title('Mean Radius Violin Plot', fontweight='bold')
ax.set_xticklabels(['Benign', 'Malignant'])

# Statistics
ax = axes[1, 1]
ax.axis('off')
stats_text = f"""
MEAN RADIUS STATISTICS:
────────────────────────
Benign:
  Mean: {df[df['target']==0]['feature_1'].mean():.3f}
  Median: {df[df['target']==0]['feature_1'].median():.3f}
  Std: {df[df['target']==0]['feature_1'].std():.3f}

Malignant:
  Mean: {df[df['target']==1]['feature_1'].mean():.3f}
  Median: {df[df['target']==1]['feature_1'].median():.3f}
  Std: {df[df['target']==1]['feature_1'].std():.3f}

Separation: {df[df['target']==1]['feature_1'].mean() - df[df['target']==0]['feature_1'].mean():.3f}
"""
ax.text(0.1, 0.5, stats_text, transform=ax.transAxes, fontsize=11,
        verticalalignment='center', family='monospace')

plt.suptitle('Mean Radius Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 12. Mean Texture Analysis (Feature 2)

# %%
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
ax = axes[0]
for target in [0, 1]:
    subset = df[df['target'] == target]['feature_2']
    ax.hist(subset, alpha=0.7, label=f'Target {target}', bins=20,
            color=['#4ECDC4', '#FF6B6B'][target])
ax.set_xlabel('Mean Texture')
ax.set_ylabel('Frequency')
ax.set_title('Mean Texture Distribution', fontweight='bold')
ax.legend(['Benign', 'Malignant'])

# Boxplot
ax = axes[1]
df.boxplot(column='feature_2', by='target', ax=ax)
ax.set_title('Mean Texture Boxplot', fontweight='bold')
ax.set_xlabel('Tumor Type')
ax.set_ylabel('Mean Texture')
ax.set_xticklabels(['Benign', 'Malignant'])

plt.suptitle('Mean Texture Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 13. Mean Perimeter Analysis (Feature 3)

# %%
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
ax = axes[0]
for target in [0, 1]:
    subset = df[df['target'] == target]['feature_3']
    ax.hist(subset, alpha=0.7, label=f'Target {target}', bins=20,
            color=['#4ECDC4', '#FF6B6B'][target])
ax.set_xlabel('Mean Perimeter')
ax.set_ylabel('Frequency')
ax.set_title('Mean Perimeter Distribution', fontweight='bold')
ax.legend(['Benign', 'Malignant'])

# Boxplot
ax = axes[1]
df.boxplot(column='feature_3', by='target', ax=ax)
ax.set_title('Mean Perimeter Boxplot', fontweight='bold')
ax.set_xlabel('Tumor Type')
ax.set_ylabel('Mean Perimeter')
ax.set_xticklabels(['Benign', 'Malignant'])

plt.suptitle('Mean Perimeter Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 14. Mean Area Analysis (Feature 4)

# %%
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
ax = axes[0]
for target in [0, 1]:
    subset = df[df['target'] == target]['feature_4']
    ax.hist(subset, alpha=0.7, label=f'Target {target}', bins=20,
            color=['#4ECDC4', '#FF6B6B'][target])
ax.set_xlabel('Mean Area')
ax.set_ylabel('Frequency')
ax.set_title('Mean Area Distribution', fontweight='bold')
ax.legend(['Benign', 'Malignant'])

# Boxplot
ax = axes[1]
df.boxplot(column='feature_4', by='target', ax=ax)
ax.set_title('Mean Area Boxplot', fontweight='bold')
ax.set_xlabel('Tumor Type')
ax.set_ylabel('Mean Area')
ax.set_xticklabels(['Benign', 'Malignant'])

plt.suptitle('Mean Area Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 15. Mean Smoothness Analysis (Feature 5)

# %%
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
ax = axes[0]
for target in [0, 1]:
    subset = df[df['target'] == target]['feature_5']
    ax.hist(subset, alpha=0.7, label=f'Target {target}', bins=20,
            color=['#4ECDC4', '#FF6B6B'][target])
ax.set_xlabel('Mean Smoothness')
ax.set_ylabel('Frequency')
ax.set_title('Mean Smoothness Distribution', fontweight='bold')
ax.legend(['Benign', 'Malignant'])

# Boxplot
ax = axes[1]
df.boxplot(column='feature_5', by='target', ax=ax)
ax.set_title('Mean Smoothness Boxplot', fontweight='bold')
ax.set_xlabel('Tumor Type')
ax.set_ylabel('Mean Smoothness')
ax.set_xticklabels(['Benign', 'Malignant'])

plt.suptitle('Mean Smoothness Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 16. Correlation Heatmap (Top 10 Features)

# %%
# Select first 10 features + target
top_features = df.columns[:10].tolist() + ['target']
corr = df[top_features].corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, linewidths=0.5,
            cbar_kws={"shrink": 0.8})
plt.title('Breast Cancer - Feature Correlation (Top 10)', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 17. Feature Correlation with Target

# %%
correlations = df.corr()['target'].drop('target').sort_values(ascending=False)

plt.figure(figsize=(12, 8))
colors = ['#2E86AB' if x > 0 else '#A23B72' for x in correlations.values[:20]]
correlations.head(20).plot(kind='bar', color=colors)
plt.title('Top 20 Features - Correlation with Malignancy', fontweight='bold', fontsize=14)
plt.xlabel('Features')
plt.ylabel('Correlation with Target')
plt.xticks(rotation=45, ha='right')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

print("📊 TOP 5 FEATURES (Highest Correlation):")
for feat, corr in correlations.head(5).items():
    print(f"   {feat}: {corr:.4f}")

In [ ]:
# %% [markdown]
# ## 18. Pairplot of Top 4 Features

# %%
top_4_features = correlations.head(4).index.tolist() + ['target']
sns.pairplot(df[top_4_features], hue='target', 
             palette=['#4ECDC4', '#FF6B6B'],
             diag_kind='kde', plot_kws={'alpha': 0.6})
plt.suptitle('Pairplot of Top 4 Discriminating Features', y=1.02, fontsize=16, fontweight='bold')
plt.show()

In [ ]:
# %% [markdown]
# ## 19. 3D Visualization of Top 3 Features

# %%
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

top_3 = correlations.head(3).index.tolist()
colors = ['#4ECDC4' if x == 0 else '#FF6B6B' for x in df['target']]

scatter = ax.scatter(df[top_3[0]], df[top_3[1]], df[top_3[2]], 
                     c=colors, alpha=0.6, s=50)

ax.set_xlabel(top_3[0], fontsize=10)
ax.set_ylabel(top_3[1], fontsize=10)
ax.set_zlabel(top_3[2], fontsize=10)
ax.set_title('3D Visualization - Top 3 Features', fontweight='bold', fontsize=14)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#4ECDC4', label='Benign'),
                   Patch(facecolor='#FF6B6B', label='Malignant')]
ax.legend(handles=legend_elements, loc='upper right')

plt.show()

In [ ]:
# %% [markdown]
# ## 20. Data Preprocessing

# %%
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split features and target
X = df.drop('target', axis=1)
y = df['target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 BEFORE SCALING:")
print(f"   Training set: {X_train.shape}")
print(f"   Testing set:  {X_test.shape}")
print(f"   Class 0: {(y_train==0).sum()} ({(y_train==0).sum()/len(y_train)*100:.1f}%)")
print(f"   Class 1: {(y_train==1).sum()} ({(y_train==1).sum()/len(y_train)*100:.1f}%)")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✅ Scaling complete")
print(f"✅ Training set scaled: {X_train_scaled.shape}")
print(f"✅ Testing set scaled:  {X_test_scaled.shape}")

In [ ]:
# %% [markdown]
# ## 21. Training: Logistic Regression

# %%
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)
lr_pred = lr_model.predict(X_test_scaled)
lr_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

print("✅ Logistic Regression trained")
print(f"   Training accuracy: {lr_model.score(X_train_scaled, y_train):.4f}")

In [ ]:
# %% [markdown]
# ## 22. Training: Support Vector Machine

# %%
from sklearn.svm import SVC

svm_model = SVC(probability=True, random_state=42)
svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)
svm_proba = svm_model.predict_proba(X_test_scaled)[:, 1]

print("✅ SVM trained")
print(f"   Training accuracy: {svm_model.score(X_train_scaled, y_train):.4f}")

In [ ]:
# %% [markdown]
# ## 23. Training: Random Forest

# %%
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_scaled, y_train)
rf_pred = rf_model.predict(X_test_scaled)
rf_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

print("✅ Random Forest trained")
print(f"   Training accuracy: {rf_model.score(X_train_scaled, y_train):.4f}")

In [ ]:
# %% [markdown]
# ## 24. Training: XGBoost

# %%
import xgboost as xgb

xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
xgb_model.fit(X_train_scaled, y_train)
xgb_pred = xgb_model.predict(X_test_scaled)
xgb_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]

print("✅ XGBoost trained")
print(f"   Training accuracy: {xgb_model.score(X_train_scaled, y_train):.4f}")

In [ ]:
# %% [markdown]
# ## 25. Model Comparison - All Metrics

# %%
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

models = ['Logistic Regression', 'SVM', 'Random Forest', 'XGBoost']
predictions = [lr_pred, svm_pred, rf_pred, xgb_pred]
probabilities = [lr_proba, svm_proba, rf_proba, xgb_proba]

results = []
for name, pred, proba in zip(models, predictions, probabilities):
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, pred),
        'Precision': precision_score(y_test, pred),
        'Recall': recall_score(y_test, pred),
        'F1-Score': f1_score(y_test, pred),
        'ROC-AUC': roc_auc_score(y_test, proba)
    })

results_df = pd.DataFrame(results).round(4)
print("📊 MODEL COMPARISON:")
display(results_df)

best_model = results_df.loc[results_df['F1-Score'].idxmax(), 'Model']
best_f1 = results_df.loc[results_df['F1-Score'].idxmax(), 'F1-Score']
print(f"\n🏆 BEST MODEL: {best_model} with F1-Score = {best_f1:.4f}")

In [ ]:
# %% [markdown]
# ## 26. Confusion Matrix (Best Model)

# %%
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

best_idx = models.index(best_model)
best_pred = predictions[best_idx]
cm = confusion_matrix(y_test, best_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Benign', 'Malignant'])
disp.plot(cmap='Blues')
plt.title(f'Confusion Matrix - {best_model}', fontweight='bold', fontsize=14)
plt.grid(False)
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f"True Negatives:  {tn}  (Correctly predicted Benign)")
print(f"False Positives: {fp}  (Incorrectly predicted Malignant)")
print(f"False Negatives: {fn}  (Missed Malignant cases)")
print(f"True Positives:  {tp}  (Correctly predicted Malignant)")
print(f"\nSensitivity: {tp/(tp+fn):.4f} - Ability to detect Malignant")
print(f"Specificity: {tn/(tn+fp):.4f} - Ability to detect Benign")

In [ ]:
# %% [markdown]
# ## 27. ROC Curves Comparison

# %%
from sklearn.metrics import roc_curve

plt.figure(figsize=(10, 8))

for name, proba in zip(models, probabilities):
    fpr, tpr, _ = roc_curve(y_test, proba)
    auc = roc_auc_score(y_test, proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', alpha=0.5)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - Breast Cancer Prediction', fontweight='bold', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# %% [markdown]
# ## 28. Feature Importance Analysis

# %%
if best_model in ['Random Forest', 'XGBoost']:
    best_model_obj = [rf_model, xgb_model][models.index(best_model)-2]
    importances = best_model_obj.feature_importances_
    
    feat_imp = pd.DataFrame({
        'Feature': X.columns,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    plt.figure(figsize=(12, 10))
    top_15 = feat_imp.head(15)
    colors = plt.cm.Blues(np.linspace(0.4, 1, len(top_15)))
    plt.barh(range(len(top_15)), top_15['Importance'][::-1], color=colors[::-1])
    plt.yticks(range(len(top_15)), top_15['Feature'][::-1])
    plt.xlabel('Importance')
    plt.title(f'Top 15 Features - {best_model}', fontweight='bold', fontsize=14)
    
    for i, (_, row) in enumerate(top_15[::-1].iterrows()):
        plt.text(row['Importance'] + 0.005, i, f'{row["Importance"]:.4f}', 
                 va='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 TOP 5 MOST IMPORTANT FEATURES:")
    for i, row in feat_imp.head(5).iterrows():
        print(f"   {row['Feature']}: {row['Importance']:.4f}")

In [ ]:
# %% [markdown]
# ## 29. Save Models for Deployment

# %%
import joblib
from datetime import datetime

# Save best model
best_model_obj = [lr_model, svm_model, rf_model, xgb_model][best_idx]
model_path = f'../models/cancer_{best_model.lower().replace(" ", "_")}_v1.0.0.pkl'
joblib.dump(best_model_obj, model_path)

# Save scaler and features
joblib.dump(scaler, '../models/cancer_scaler.pkl')
joblib.dump(X.columns.tolist(), '../models/cancer_features.pkl')

print(f"✅ Best model saved: {model_path}")
print(f"✅ Scaler saved: ../models/cancer_scaler.pkl")
print(f"✅ Feature names saved: ../models/cancer_features.pkl")
print(f"📅 Saved at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# %% [markdown]
# ## 30. Model Metadata

# %%
import json

metadata = {
    'model_name': best_model,
    'version': '1.0.0',
    'training_date': datetime.now().isoformat(),
    'dataset': 'Wisconsin Breast Cancer Database',
    'samples': len(df),
    'features': len(X.columns),
    'feature_list': X.columns.tolist()[:10] + ['...'],  # First 10 features
    'metrics': {
        'accuracy': float(results_df.loc[best_idx, 'Accuracy']),
        'precision': float(results_df.loc[best_idx, 'Precision']),
        'recall': float(results_df.loc[best_idx, 'Recall']),
        'f1_score': float(results_df.loc[best_idx, 'F1-Score']),
        'roc_auc': float(results_df.loc[best_idx, 'ROC-AUC'])
    },
    'preprocessing': {
        'scaler': 'StandardScaler',
        'test_size': 0.2,
        'random_state': 42
    }
}

with open('../models/cancer_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Model metadata saved")

In [ ]:
# %% [markdown]
# ## 31. Test Single Prediction

# %%
# Take first test sample
sample = X_test_scaled[0].reshape(1, -1)
actual = y_test.iloc[0]
pred = best_model_obj.predict(sample)[0]
proba = best_model_obj.predict_proba(sample)[0]

print("🔍 SINGLE PREDICTION TEST:")
print("="*60)
print(f"Actual:      {'MALIGNANT' if actual == 1 else 'BENIGN'}")
print(f"Predicted:   {'MALIGNANT' if pred == 1 else 'BENIGN'}")
print(f"Probability: Benign: {proba[0]:.4f}, Malignant: {proba[1]:.4f}")
print(f"Confidence:  {max(proba)*100:.2f}%")
print(f"Risk Level:  {'HIGH' if pred == 1 else 'LOW'}")
print("="*60)

if pred == actual:
    print("✅ Prediction CORRECT")
else:
    print("❌ Prediction INCORRECT")

In [ ]:
# %% [markdown]
# ## 32. Export Results to CSV

# %%
results_df.to_csv('../reports/cancer_model_comparison.csv', index=False)
print("✅ Results exported to reports/cancer_model_comparison.csv")

# Display styled table
styled = results_df.style.background_gradient(cmap='Blues', subset=['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC'])
styled

In [ ]:
# %% [markdown]
# ## 33. Pipeline Complete - Final Summary

# %%
summary = f"""
╔══════════════════════════════════════════════════════════════════╗
║              BREAST CANCER PREDICTION - PIPELINE COMPLETE        ║
╠══════════════════════════════════════════════════════════════════╣
║  Dataset:        Wisconsin Breast Cancer                         ║
║  Patients:       {len(df):,}                                      ║
║  Features:       {len(X.columns)}                                 ║
║  Benign cases:   {target_dist[0]} ({target_pct[0]:.1f}%)         ║
║  Malignant cases: {target_dist[1]} ({target_pct[1]:.1f}%)        ║
║  Best Model:     {best_model}                                     ║
║  F1-Score:       {results_df.loc[best_idx, 'F1-Score']:.4f}      ║
║  Accuracy:       {results_df.loc[best_idx, 'Accuracy']:.4f}      ║
║  Precision:      {results_df.loc[best_idx, 'Precision']:.4f}     ║
║  Recall:         {results_df.loc[best_idx, 'Recall']:.4f}        ║
║  ROC-AUC:        {results_df.loc[best_idx, 'ROC-AUC']:.4f}       ║
║  Model saved:    models/cancer_{best_model.lower().replace(' ', '_')}_v1.0.0.pkl  ║
║  Reports:        reports/cancer_model_comparison.csv             ║
╚══════════════════════════════════════════════════════════════════╝
"""

print(summary)

with open('../reports/cancer_summary.txt', 'w') as f:
    f.write(summary)

In [ ]:
# %% [markdown]
# ## 34. Ready for Deployment

# %%
print("""
🚀 DEPLOYMENT INSTRUCTIONS:

1. Start Breast Cancer Prediction API:
   $ uvicorn api.main:app --reload

2. Test API endpoint:
   curl -X POST http://localhost:8000/predict/cancer \\
        -H "Content-Type: application/json" \\
        -d '{
            "feature_1": 14.2,
            "feature_2": 19.5,
            "feature_3": 92.5,
            "feature_4": 650.2,
            "feature_5": 0.095,
            ... (all 30 features)
        }'

3. Expected response:
   {
     "disease": "Breast Cancer",
     "prediction": 0,
     "probability": 0.98,
     "risk_level": "Low",
     "confidence": 0.98,
     "tumor_type": "Benign"
   }

🎉 Breast cancer prediction model is ready for production!
""")

In [ ]:
# %% [markdown]
# ## 35. Multi-Disease System Complete

# %%
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                                                                      ║
║     🏆 MULTI-DISEASE PREDICTION SYSTEM - COMPLETE 🏆                ║
║                                                                      ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║   ✅ HEART DISEASE PIPELINE                                          ║
║      - 303 patients, 13 features                                     ║
║      - Best model: [From heart notebook]                             ║
║      - F1-Score: [From heart notebook]                               ║
║                                                                      ║
║   ✅ DIABETES PIPELINE                                                ║
║      - 768 patients, 8 features                                      ║
║      - Best model: {best_model}                                      ║
║      - F1-Score: {results_df.loc[best_idx, 'F1-Score']:.4f}          ║
║                                                                      ║
║   ✅ BREAST CANCER PIPELINE                                           ║
║      - 569 patients, 30 features                                     ║
║      - Best model: {best_model}                                      ║
║      - F1-Score: {results_df.loc[best_idx, 'F1-Score']:.4f}          ║
║                                                                      ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║   📁 MODELS SAVED:                                                   ║
║      - /models/heart_*.pkl                                           ║
║      - /models/diabetes_*.pkl                                        ║
║      - /models/cancer_*.pkl                                          ║
║                                                                      ║
║   📊 REPORTS SAVED:                                                  ║
║      - /reports/heart_*.csv                                          ║
║      - /reports/diabetes_*.csv                                       ║
║      - /reports/cancer_*.csv                                         ║
║                                                                      ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║   🚀 NEXT STEPS:                                                     ║
║    1. Start the API server: uvicorn api.main:app --reload           ║
║    2. Access API docs: http://localhost:8000/docs                   ║
║    3. Test predictions via API or frontend                           ║
║    4. Deploy with Docker: docker-compose up                          ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
""")